# quality_assurance

> In progress, including development of diagnostics to be moved elsewhere when complete.

In [ ]:
# |default_exp quality_assurance

In [ ]:
# | hide
from nbdev.showdoc import show_doc

In [ ]:
# |export

import time
from abc import ABC, abstractmethod

import dask
import pandas as pd
import xarray as xr

To write checks of the data, we first create a subclass of `Diagnostics`.

In [ ]:
# |export


class Diagnostics(ABC):
    """An abstract class to be subclassed to perform specific diagnostic checks.

    A subclass should perform a set of checks, implemented in a method named `tests`.

    Calling the method `run` will combine and compute the tests, putting the results
    as a single boolean `DataArray` the `detail` attribute for further analysis.

    Once `run` has been called, a `test_descriptions` will also be available, and
    summaries of the test results can be created using the `summary` method.
    """

    def __init__(self):
        """Initialise the diagnostic.

        If the subclass has options, these may be added like this:
        ```
        def __init__(
            self,
            tolerance: float = 0.0,  # the tolerance for the test
        ):
            self.tolerance = tolerance
            super().__init__()
        ```
        then `self.tolerance` can be used in the implementation of `tests`.
        """
        self.test_descriptions = None
        self.detail = None

    def run(self, **kwargs) -> xr.DataArray:
        """Compute the results of the tests.

        The `kwargs` are passed to `qagmire.data.read_*` functions to obtain the data
        for the tests.
        """
        tests = self.tests(**kwargs)
        test_names = [t["name"] for t in tests]
        test_desc = [t["description"] for t in tests]
        self.test_descriptions = dict(zip(test_names, test_desc))
        test_array = [t["test"] for t in tests]
        detail = xr.concat(test_array, pd.Index(test_names, name="test"))
        start = time.perf_counter()
        self.detail = dask.compute(detail)[0]
        dt = time.perf_counter() - start
        print(f"Tests took {dt:.2f} s to perform.")
        for name, desc in self.test_descriptions.items():
            print(f"{name}:\n    {desc}")

    @abstractmethod
    def tests(self, **kwargs):
        """Return the tests to be performed.

        Implementations of this method must pass `kwargs` to `qagmire.data.read_*` functions
        as necessary to obtain the data for the tests.

        This method must returns a list of dictionaries with the structure:
        ```
        [
            {
                "name": "a_short_name",
                "description": "The question that the test answers",
                "test": test_dataset,
            },
            ...
        ]
        ```
        where each `test_dataset` should be a boolean `xr.DataArray` of the same shape, giving
        the results of running the test on the data defined by `kwargs`.
        """
        return [
            {
                "name": "a_short_name",
                "description": "The question that the test answers",
                "test": None,
            },
        ]

    def summary(
        self,
        by: str | None = None,  # optionally sum element dims except for this one
        show_passed_tests=False,  # if `True`, then passed tests are included
        show_passed_elements=False,  # if `True`, then passed elements are included
        sort_by_total_fails=True,  # if `False`, then keep in original order
        show_failure_count=True,  # if `False`, then omit the count of failures per row
        show_only_failure_count=False,  # if `True`, then only show the count of failures
        per_test=False,  # if `True`, then transpose output, such that each row is a test
        top: int | None = None,  # optionally limit to at most `top` elements
    ) -> pd.DataFrame:
        """Return a summary of the test failures."""
        if self.detail is None:
            print("You need to call `run` first.")
            return None
        else:
            detail = self.detail
        if by is not None:
            detail = detail.sum(dim=[d for d in detail.dims if d not in ("test", by)])
        # convert to a dataframe and drop any extra coordinates
        df = detail.to_dataframe(name="failed")[["failed"]]
        df = df.unstack() if per_test else df.unstack("test")
        if (not show_passed_tests and not per_test) or (
            not show_passed_elements and per_test
        ):
            df = df.loc[:, df.any(axis="rows")]
        if (not show_passed_elements and not per_test) or (
            not show_passed_tests and per_test
        ):
            df = df.loc[df.loc[:, "failed"].any(axis="columns")]
        df.loc[:, "total fails"] = df.sum(axis="columns")
        if sort_by_total_fails:
            df = df.sort_values("total fails", ascending=False)
        if not (show_failure_count or show_only_failure_count):
            df = df.drop(columns="total fails")
        if show_only_failure_count:
            df = df.drop(columns="failed")
        if top is not None:
            df = df.iloc[:top]
        return df

    def summary_per_test(
        self,
        by: str | None = None,  # optionally sum element dims except for this one
    ) -> pd.DataFrame:
        """Return a per-test summary of the test outcomes in `detail`."""
        return self.summary(
            by=by,
            per_test=True,
            show_passed_tests=True,
            show_only_failure_count=True,
        )

    def full_summary(
        self,
        by: str | None = None,  # optionally sum element dims except for this one
    ) -> pd.DataFrame:
        """Return a full summary of the test outcomes in `detail`."""
        return self.summary(
            by=by,
            show_passed_tests=True,
            show_passed_elements=True,
            sort_by_total_fails=False,
            show_failure_count=False,
            top=None,
        )

In this subclass we need to implement the `tests` method.

In [ ]:
# |hide
show_doc(Diagnostics.tests)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/quality_assurance.py#L75){target="_blank" style="float:right; font-size:smaller"}

### Diagnostics.tests

>      Diagnostics.tests (**kwargs)

Return the tests to be performed.

Implementations of this method must pass `kwargs` to `qagmire.data.read_*` functions
as necessary to obtain the data for the tests.

This method must returns a list of dictionaries with the structure:
```
[
    {
        "name": "a_short_name",
        "description": "The question that the test answers",
        "test": test_dataset,
    },
    ...
]
```
where each `test_dataset` should be a boolean `xr.DataArray` of the same shape, giving
the results of running the test on the data defined by `kwargs`.

These tests are executed by calling the `run` method.

In [ ]:
# |hide
show_doc(Diagnostics.run)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/quality_assurance.py#L55){target="_blank" style="float:right; font-size:smaller"}

### Diagnostics.run

>      Diagnostics.run (**kwargs)

Compute the results of the tests.

The `kwargs` are passed to `qagmire.data.read_*` functions to obtain the data
for the tests.

The `summary` method outputs a pandas DataFrame summary of the test outcomes, by default this shows only failed tests and elements (e.g. OBs or exposures) with the most failures.

In [ ]:
# |hide
show_doc(Diagnostics.summary)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/quality_assurance.py#L103){target="_blank" style="float:right; font-size:smaller"}

### Diagnostics.summary

>      Diagnostics.summary (by:str|None=None, show_passed_tests=False,
>                           show_passed_elements=False,
>                           sort_by_total_fails=True, show_failure_count=True,
>                           show_only_failure_count=False, per_test=False,
>                           top:int|None=None)

Return a summary of the test failures.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| by | str \| None | None | optionally sum element dims except for this one |
| show_passed_tests | bool | False | if `True`, then passed tests are included |
| show_passed_elements | bool | False | if `True`, then passed elements are included |
| sort_by_total_fails | bool | True | if `False`, then keep in original order |
| show_failure_count | bool | True | if `False`, then omit the count of failures per row |
| show_only_failure_count | bool | False | if `True`, then only show the count of failures |
| per_test | bool | False | if `True`, then transpose output, such that each row is a test |
| top | int \| None | None | optionally limit to at most `top` elements |
| **Returns** | **DataFrame** |  |  |

See the [diagnostics](diagnostics) submodule for example tests.

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()